In [219]:
import numpy as np
import h5py
import simulator3
import compartment
import electrodiffusion
import common

### Reading old file

In [277]:

base_file_name = "T5"
new_file_name = "T5_stitch"



with h5py.File(base_file_name, mode='r') as hdf_old:
    C = hdf_old.get('COMPARTMENTS')
    C_group_arr = []
    comp_names_arr = list(C.keys())

    T = hdf_old.get('TIMING')
    total_t = T.get('TOTAL_T')[()]
    intervals = T.get('INTERVALS')[()]
    dt = T.get("DT")[()]
    total_steps = total_t / dt
    interval_step = total_steps / intervals
    interval_arr = [round(interval_step * i) for i in range(intervals)]
    
    print('Total_time: ' + str(total_t))
    print('Total_intervals: ' + str(intervals))
    
    
    for e in comp_names_arr:

        comp_name = e
        current_comp = C.get(comp_name)
    
        try:
            last_interval = 0
            for j in range(len(list(current_comp.keys()))):
                #dataset = C_group.get(str(interval_arr[j]))
                last_interval += 1
            print(e + ' last interval = ' + str(last_interval))
        except:
            print(last_interval)

            



Total_time: 120
Total_intervals: 1000
Comp1 last interval = 932
Comp2 last interval = 932
Comp3 last interval = 932
Comp4 last interval = 932
Comp5 last interval = 932
Comp6 last interval = 932
Comp7 last interval = 932
Comp8 last interval = 932
Comp9 last interval = 932


In [278]:
new_sim = simulator3.Simulator(new_file_name)
# creates a new file in C://

new_file_name = "C:\ T5_stitch"
with h5py.File(new_file_name, mode='r+') as hdf_new:
    
    hdf_new.filename 
    with h5py.File(base_file_name, mode='r') as hdf_old:
        groups = hdf_old.keys()
        groups_list = list(groups) 
        print(groups_list)
        for i in range(len(groups_list)):
            hdf_new_group = hdf_new.get(groups_list[i])
            group = hdf_old.get(groups_list[i])
            subgroups = group.keys()
            subgroups_list = list(subgroups)
            for j in range(len(subgroups_list)):
                subgroup = group.get(subgroups_list[j])
                print(subgroup)
                hdf_old.copy(subgroup, hdf_new[groups_list[i]])
            
        
            #hdf_old.copy(groups[i],group)
            
            #hdf_new.create_group(group)
        #hdf_new.create_group('COMPARTMENTS')
   

simulation file ('T5_stitch') created in base directory
['COMPARTMENTS', 'CURRENT-SETTINGS', 'ELECTRODIFFUSION', 'SYNAPSE-SETTINGS', 'TIMING', 'X-FLUX-SETTINGS']
<HDF5 group "/COMPARTMENTS/Comp1" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp2" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp3" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp4" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp5" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp6" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp7" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp8" (932 members)>
<HDF5 group "/COMPARTMENTS/Comp9" (932 members)>
<HDF5 group "/ELECTRODIFFUSION/Comp1 -> Comp2" (931 members)>
<HDF5 group "/ELECTRODIFFUSION/Comp2 -> Comp3" (931 members)>
<HDF5 group "/ELECTRODIFFUSION/Comp3 -> Comp4" (931 members)>
<HDF5 group "/ELECTRODIFFUSION/Comp4 -> Comp5" (931 members)>
<HDF5 group "/ELECTRODIFFUSION/Comp5 -> Comp6" (931 members)>
<HDF5 group "/ELECTRODIFFUSION/Comp6 -> Comp7" (931 members)>
<HDF5 group "/ELECTRODIFF

### Setting up new simulation

In [279]:

with h5py.File(new_file_name, mode='a') as hdf:
    C = hdf.get('COMPARTMENTS')
    C_group_arr = []
    comp_names_arr = list(C.keys())

    T = hdf.get('TIMING')
    total_t = T.get('TOTAL_T')[()]
    intervals = T.get('INTERVALS')[()]
    dt = T.get("DT")[()]
    total_steps = total_t / dt
    interval_step = total_steps / intervals
    interval_arr = [round(interval_step * i) for i in range(intervals)]
    
    starting_sa_arr = []
    starting_na_arr = []

    for i in comp_names_arr:
        temp_arr = []
        comp_name = i
        current_comp = C.get(comp_name)
        
        first_time_point = str(0)
        first_dataset = current_comp.get(first_time_point)
        
        radius = first_dataset[1]
        length = first_dataset[2]
        na = first_dataset[4]
        
        starting_sa_arr.append(2*np.pi * radius* length)
        starting_na_arr.append(na)
        
        
        last_time_point = str(interval_arr[last_interval-1])
        last_dataset = current_comp.get(last_time_point)
        last_dataset = list(last_dataset)

        radius = last_dataset[1]
        length = last_dataset[2]
        na_i = last_dataset[4]
        k_i = last_dataset[5]
        cl_i = last_dataset[6]
        x_i = last_dataset[7]
        z_i = last_dataset[8]
        vm = last_dataset[-3]

        if comp_name == "Comp0(Soma)":
            comp = compartment.Compartment(comp_name, radius, length, static_sa=True, sa_value=2 * np.pi * 1e-5 * 40e-5)
            comp.set_ion_properties(na_i, k_i, cl_i, x_i, z_i, osmol_neutral_start=False, adjust_x=False)
        else:
            comp = compartment.Compartment(comp_name, radius, length, static_sa=True,
                                           sa_value=2 * np.pi * 0.5e-5 * 20e-5)
            comp.set_ion_properties(na_i, k_i, cl_i, x_i, z_i, osmol_neutral_start=False)
        # comp.set_voltages(vm)
        new_sim.comp_arr.append(comp)

    hdf.close()

### Adding sim settings

In [280]:

for i in range(len(comp_arr)):
    new_sim.comp_arr[i].sa = starting_sa_arr[i]
    new_sim.comp_arr[i].na_i_start = starting_na_arr[i]
    
new_sim.set_sa_static(static_sa=True)
new_sim.set_atpase_static(static_atpase=True)


new_sim.set_external_ion_properties()


### Adding electrodiffusion

In [281]:
new_sim.ED_on = True
new_sim.diff_constants_dict={"na": (1.33 / 2) * 1e-7, "k": (1.96 / 2) * 1e-7, "cl": (2.03 / 2) * 1e-7, "x": 0}

new_sim.ed_arr =[ ]
for e in range(len(new_sim.comp_arr)-1):
    name =new_sim.comp_arr[e].name + ' -> ' + new_sim.comp_arr[e + 1].name

    ed = electrodiffusion.Electrodiffusion(comp_a_name=new_sim.comp_arr[e].name,
                                          comp_a_length=new_sim.comp_arr[e].length,
                                          comp_b_name=new_sim.comp_arr[e + 1].name,
                                          comp_b_length=new_sim.comp_arr[e + 1].length)

    ed.set_diff_constants(new_sim.diff_constants_dict)
    new_sim.ed_arr.append(ed)


### Setting timing

In [282]:
new_sim.dt = dt
new_sim.total_t = total_t
new_sim.start_t = interval_arr[last_interval-1]*1e-6
new_sim.remaining_t = new_sim.total_t - new_sim.start_t
new_sim.total_steps = new_sim.remaining_t / new_sim.dt
new_sim.remaining_intervals = intervals - last_interval
print("Total intervals: " + str(intervals))
print("Current interval: " + str(last_interval))
print("Remaining intervals: " + str(new_sim.remaining_intervals))

print("Time remaining: " + str(new_sim.remaining_t))
print("Steps remaining: " + str(new_sim.total_steps))
new_sim.output_intervals = (0.001, 0.005, 0.01, 0.1, 0.25, 0.5, 0.75, 1)
new_sim.output_arr = [round(new_sim.output_intervals[a] * new_sim.total_steps, 0) for a in
                           range(len(new_sim.output_intervals))]
new_sim.output_arr = tuple(new_sim.output_arr)

new_sim.interval_step = new_sim.total_steps / new_sim.remaining_intervals
print("Interval_step: " + str(new_sim.interval_step))

new_sim.interval_arr = [round(new_sim.interval_step * i) for i in range(new_sim.remaining_intervals)]
new_sim.interval_arr = tuple(new_sim.interval_arr)


Total intervals: 1000
Current interval: 932
Remaining intervals: 68
Time remaining: 8.280000000000001
Steps remaining: 8280000.000000002
Interval_step: 121764.70588235297


### Flux settings

In [283]:
for i in new_sim.comp_arr:
    print(i.name)

Comp1
Comp2
Comp3
Comp4
Comp5
Comp6
Comp7
Comp8
Comp9


In [284]:
print(new_sim.file_name)
with h5py.File(new_sim.file_name, mode='r') as hdf:
    print(list(hdf.keys()))
    C = hdf.get('COMPARTMENTS')  
    print(list(C.keys()))
      

\ T5_stitch
['COMPARTMENTS', 'CURRENT-SETTINGS', 'ELECTRODIFFUSION', 'SYNAPSE-SETTINGS', 'TIMING', 'X-FLUX-SETTINGS']
['Comp1', 'Comp2', 'Comp3', 'Comp4', 'Comp5', 'Comp6', 'Comp7', 'Comp8', 'Comp9']


In [285]:
new_sim.run_simulation()

0.1 % complete in 3.68 s
0.5 % complete in 17.93 s
1.0 % complete in 35.6 s
Estimated time to complete :59.33 minutes
10.0 % complete in 397.57 s
25.0 % complete in 959.93 s
50.0 % complete in 2031.45 s
75.0 % complete in 2899.59 s


KeyboardInterrupt: 